In [1]:
# |default_exp rag

## Install dependencies

## Make an app with Gradio

In [43]:
# |export
import ollama
import re
import gradio as gr
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama import OllamaEmbeddings
from chromadb.config import Settings
from chromadb import Client
import chromadb
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os
import csv

from fastcore.net import urljson, HTTPError
from openai import api_key
from openai import OpenAI
import openai

In [5]:
# |export
import chromadb.utils.embedding_functions as embedding_functions
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [38]:
load_dotenv()
# deepseek_key = os.getenv('DEEPSEEK_R1_API_KEY')
gemini_key = os.getenv('GEMINI_API_KEY')

In [11]:
loader = PyMuPDFLoader("../res/employee_manual.pdf")
documents = loader.load()

In [16]:
# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [13]:
# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=deepseek_key,
# )

In [ ]:
# embedding = client.embeddings.create(
#         model="deepseek/deepseek-r1:free",
#         input='Your text string goes here'
#     )
# embedding

In [39]:
google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=gemini_key)

In [26]:
# Parallelize embedding generation
def generate_embedding(chunk):
    # return ollama_embedding_function.embed_query(chunk.page_content)
    return google_ef.embed_query(chunk.page_content)


with ThreadPoolExecutor() as executor:
    embeddings = list(executor.map(generate_embedding, chunks))

AttributeError: 'OpenAIEmbeddingFunction' object has no attribute 'embed_query'

In [35]:

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "binjian.github.io", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "binjian's digital garden", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="deepseek/deepseek-r1:free",
  messages=[
    {
      "role": "system", "content": "You are a helpful assistant.",
      "role": "user", "content": "What's Anthropic's Model context protocol?"
    }
  ]
)
print(completion.choices[0].message.content)

Anthropic's Model Context Protocol refers to the structured approach used in their AI models, particularly Claude, to manage conversational context effectively while prioritizing safety and coherence. Here's a breakdown of its key components:

1. **Context Window Management**: 
   - Claude's large context window (up to 200k tokens) allows processing lengthy inputs, but the protocol ensures efficient retention of relevant information. Techniques like attention mechanisms prioritize recent or critical dialogue segments while avoiding overload.

2. **Safety and Alignment**:
   - Rooted in **Constitutional AI**, the protocol incorporates ethical guidelines to filter harmful requests, reduce biases, and ensure outputs align with human values. It may refuse unsafe tasks or seek clarification for ambiguous queries.

3. **Multi-Turn Coherence**:
   - Maintains consistency across long conversations by tracking dialogue history, user intent, and key entities. This avoids repetitive or contradict

In [40]:
google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=gemini_key)

In [46]:
def convert_qa_to_csv(input_file, output_file):
    """
    Convert a text file with Q/A format to a CSV file.

    Args:
        input_file: Path to the input text file
        output_file: Path to the output CSV file
    """
    # Read the content of the file
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content by 'Q' marker
    qa_blocks = content.split('Q\n')

    qa_blks = [block.strip() for block in qa_blocks][1:]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [[line for line in block.split('\n') ] for block in qa_blks if block.strip()]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [blk for block in qa_blocks if block.strip() for blk in block.strip()]

    # Process each Q&A block
    qa_pairs = []
    for block in qa_blks:
        # Split the block into lines
        lines = block.strip().split('\n')

        if lines:
            # First line is the question
            question = lines[0]
            # The rest are the answer
            answer = '\n'.join(lines[1:])

            # Add the pair to our list
            qa_pairs.append([question, answer])

    # Write to CSV
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Question', 'Answer'])
        # Write Q&A pairs
        for pair in qa_pairs:
            writer.writerow(pair)
    print(f"Conversion complete. CSV file saved to {output_file}")

In [47]:

input_files = ["../res/qa_service.txt", "../res/qa_technology.txt"]
output_files = ["../res/qa_service.csv", "../res/qa_technology.csv"]
for in_f, ot_f in zip(input_files, output_files):
    convert_qa_to_csv(in_f, ot_f)
    

Conversion complete. CSV file saved to ../res/qa_service.csv
Conversion complete. CSV file saved to ../res/qa_technology.csv


In [48]:
# |export
client = chromadb.PersistentClient(path="../vdb")
# collections = [client.get_or_create_collection(name="siasun_qa_service",embedding_function=deepseek_ef),
#                 client.get_or_create_collection(name="siasun_qa_technology",embedding_function=deepseek_ef)]
collections = [client.get_or_create_collection(name="siasun_qa_service", embedding_function=google_ef),
                client.get_or_create_collection(name="siasun_qa_technology", embedding_function=google_ef)]

In [11]:
i=2
f'q{i}'

'q2'

In [49]:
# |export
for csv_file,collection in zip(output_files,collections):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            collection.add(
                documents = row,
                metadatas = [{"source": "question"}, {"source": "answer"}],
                ids = [f"{collection.name}_q{i}", f"{collection.name}_a{i}"]
            )

TypeError: RetryError.__init__() missing 1 required positional argument: 'cause'

In [19]:
#|export
queries=["你们的产品需要多久维护一次?","我怎么设置机器人的安全工作区域?"]
queries[0]

'你们的产品需要多久维护一次?'

In [10]:
results = collections[0].query(
    query_texts=queries,
    n_results=4
)

In [11]:
results

{'ids': [['siasun_qa_service_q2',
   'siasun_qa_service_a9',
   'siasun_qa_service_q21',
   'siasun_qa_service_a10'],
  ['siasun_qa_service_q1',
   'siasun_qa_service_a19',
   'siasun_qa_service_q22',
   'siasun_qa_service_a18']],
 'embeddings': None,
 'documents': [['你们的产品，多久需要维护一次？维护保养内容有哪些？',
   '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。',
   '宏作业是干什么用的？',
   '新松有标准的视觉通讯协议，视觉厂家可以按照此协议进行开发适配。当前适配过的品牌有，沈阳自动化所，欧姆龙、海康、梅卡曼德、视比特、基恩士。'],
  ['我们的人员调试不熟练，进度慢，你们厂家能负责调试吗？',
   '机器人打精度用到的开关。',
   '机器人可以存储多少个作业？',
   '零位设定，是机器人在零位时的码盘值，码盘输入、码盘输出是以零位时为零开始计数的码盘值。']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}],
  [{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}]],
 'distances': [[0.35835238473584624,
   0.8026765812953612,
   0.8398664268040978,
   0.8905950951966901],
  [0.5583867931983068,
   0.5964258746641342,
   0.6352793698255077,
   0.73287700

In [12]:
# results['metadatas'][0] #[0]['source']
results['documents'][0][0]

'你们的产品，多久需要维护一次？维护保养内容有哪些？'

In [13]:
colls = client.list_collections()
colls[0]

'siasun_qa_service'

In [47]:
queries1 = ['你们的产品需要多久维护一次?','宏作业有什么用?']

In [48]:

results = collections[0].query(
    query_texts=queries1,
    n_results=4
)
results


{'ids': [['siasun_qa_service_q2',
   'siasun_qa_service_a9',
   'siasun_qa_service_q21',
   'siasun_qa_service_a10'],
  ['siasun_qa_service_q16',
   'siasun_qa_service_a20',
   'siasun_qa_service_q13',
   'siasun_qa_service_a7']],
 'embeddings': None,
 'documents': [['你们的产品，多久需要维护一次？维护保养内容有哪些？',
   '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。',
   '宏作业是干什么用的？',
   '新松有标准的视觉通讯协议，视觉厂家可以按照此协议进行开发适配。当前适配过的品牌有，沈阳自动化所，欧姆龙、海康、梅卡曼德、视比特、基恩士。'],
  ['离线建模使用是样册的标准杆长，但实际机器人内有杆长补偿，这样建模对离线下发的点位准确性是否有影响？',
   '有，离线接口库。',
   '欧拉角是什么？有什么用处？新松机器人的欧拉角顺序是什么？',
   '还有其它报警跟随，还需要查看其它报警来确认原因。弧焊与离线都会产生离线运动失败的报警。']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}],
  [{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}]],
 'distances': [[0.35835238473584624,
   0.8026765812953612,
   0.8398664268040978,
   0.8905950951966901],
  [0.5112318120344393,
   0.5778327751539375,
  

In [14]:

# question = "你们的产品需要多久维护一次?"
# question = "你们在售前评估上，如何帮助到我们?"
answers = []
for collection in collections:
    results = collection.query(
        query_texts=queries,
        n_results=4
    )
    docs = []
    for i,metadata in enumerate(results['metadatas'][0]):
        if metadata['source'] == 'question':
            docs.append({'id': results['ids'][0][i],
                         'document': results['documents'][0][i],
                         'distance':results['distances'][0][i]})
    df = pd.DataFrame(docs)
    answers.append(df)
df_answers = pd.concat(answers, axis=0,ignore_index=True)
# df_answers = pd.stack(answers, axis=2)

In [15]:
df_answers.loc[df_answers['distance'].idxmin()]

id          siasun_qa_technology_q10
document         你们在售前评估上，能提供什么样的帮助？
distance                    0.376322
Name: 2, dtype: object

In [16]:

id_q = df_answers.loc[df_answers['distance'].idxmin()]['id']
id_q

'siasun_qa_technology_q10'

In [17]:

id_a_list = id_q.split('_')
id_a_list[-1] = id_a_list[-1].replace('q','a')
id_a_list

['siasun', 'qa', 'technology', 'a10']

In [18]:
id_a = '_'.join(id_a_list)
id_a

'siasun_qa_technology_a10'

In [19]:
coll_idx = 0 if id_a_list[-2] == 'service' else 1
coll_idx

1

In [20]:
answer = collections[coll_idx].get(id_a)
answer['documents']

['部分情况可提供现场技术指导，提供成功应用案例经验支持，仿真模拟场景，评估负载等风险，提供机械、电气、软件接口对接。']

In [21]:
best_answer = df_answers.loc[df_answers['distance'].idxmin()]

In [42]:
def qa(questions:list[str], collections:list[chromadb.Collection]=collections):
    matched_questions = []
    for collection in collections:
        results = collection.query(
            query_texts=questions,
            n_results=4
        )
        docs = []
        for i,metadata in enumerate(results['metadatas'][0]):
            if metadata['source'] == 'question':
                docs.append({'id': results['ids'][0][i],
                                'document': results['documents'][0][i], 
                                'distance':results['distances'][0][i]})
        df = pd.DataFrame(docs)
        matched_questions.append(df)

    df_matched_questions = pd.concat(matched_questions,axis=0,ignore_index=True)
    best_match_q_id = df_matched_questions.loc[df_matched_questions['distance'].idxmin()]['id']
    id_a_list = best_match_q_id.split('_')
    id_a_list[-1] = id_a_list[-1].replace('q','a')
    id_a = '_'.join(id_a_list)
    coll_idx = 0 if id_a_list[-2] == 'service' else 1
    best_answer = collections[coll_idx].get(id_a)['documents']
    res_text = best_answer[0]
    return res_text
    # question =
    # return answers

In [34]:

question = "你们的产品需要多久维护一次?"
res = qa(question)
res


['根据机器人的型号和实际使用情况，制定机器人的保养计划,一般分为日常、3 个月、6 个月、1 年期的维护保养。\n需要对机器人进行日常点检和定期维护保养，点检工作主要检查设备是否存在漏油、异响、异常震动、异常报警；定期维护保养主要对油脂、线束护套、风扇、电机接头等易损位置进行检查，并定期更换润滑油。具体检验项目及维护周期详见安装维护手册。']

In [36]:
print(res[0])

根据机器人的型号和实际使用情况，制定机器人的保养计划,一般分为日常、3 个月、6 个月、1 年期的维护保养。
需要对机器人进行日常点检和定期维护保养，点检工作主要检查设备是否存在漏油、异响、异常震动、异常报警；定期维护保养主要对油脂、线束护套、风扇、电机接头等易损位置进行检查，并定期更换润滑油。具体检验项目及维护周期详见安装维护手册。


In [43]:
# |export
iface = gr.Interface(fn=qa, inputs=gr.Text(value="多久维护一次产品?"), outputs="text")
iface.launch(width=500,share=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [41]:
# this is only necessary in a notebook
iface.close()

Closing server running on port: 7860


## Create a `requirements.txt` file

In [26]:
%%writefile ../requirements.txt
fastcore

Overwriting ../requirements.txt


In [27]:
# | hide
import nbdev

nbdev.nbdev_export()

In [28]:
# |default_exp data_preprocessing

## Convert this notebook into a Gradio app

In [29]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [30]:
# | hide
import nbdev

nbdev.nbdev_export()